<a href="https://colab.research.google.com/github/DinhQuocTuan0108/Bai-tap/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Khai báo và sử dụng các thư viện sau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import validation 
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
%cd /content/drive/MyDrive/Colab_Notebooks/project/


/content/drive/MyDrive/Colab_Notebooks/project


In [3]:
# Tạo ra class ImageDataGenerator để chỉnh sửa ảnh 
train_datagen=ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

In [4]:
# Tạo bộ dữ liệu training set
training_set=train_datagen.flow_from_directory('/content/drive/MyDrive/Colab_Notebooks/project/TRAIN',target_size=(150,150),batch_size=32, class_mode='categorical')

Found 7439 images belonging to 8 classes.


In [5]:
# Tạo bộ dữ liệu validation
validation_set=train_datagen.flow_from_directory('/content/drive/MyDrive/Colab_Notebooks/project/VALIDATION',target_size=(150,150),batch_size=32, class_mode='categorical')

Found 7439 images belonging to 8 classes.


In [6]:
# Các nhãn có trong bộ dữ liệu training set
training_set.class_indices

{'Age related Macular Degeneration': 0,
 'Cataract': 1,
 'Diabetes': 2,
 'Glaucoma': 3,
 'Hypertension': 4,
 'Normal': 5,
 'Other diseases_abnormalities': 6,
 'Pathological Myopia': 7}

In [7]:
# Các nhãn có trong bộ dữ liệu validation
validation_set.class_indices

{'Age related Macular Degeneration': 0,
 'Cataract': 1,
 'Diabetes': 2,
 'Glaucoma': 3,
 'Hypertension': 4,
 'Normal': 5,
 'Other diseases_abnormalities': 6,
 'Pathological Myopia': 7}

In [8]:
# Tạo ra mạng CNN để train mô hình
model=Sequential()
# Tích chập 32 lần với mỗi lần là 3 hàng 3 cột
model.add(Conv2D(32,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same',input_shape=(150,150,3))) 
model.add(Conv2D(32,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# Lấy phần tử lớn nhất ở trong 2 hàng và 2 cột
model.add(MaxPooling2D(2,2))
# Tích chập 64 lần với mỗi lần là 3 hàng 3 cột
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# Lấy phần tử lớn nhất ở trong 2 hàng và 2 cột
model.add(MaxPooling2D(2,2))
# Tích chập 128 lần với mỗi lần là 3 hàng 3 cột
model.add(Conv2D(128,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
model.add(Conv2D(128,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# Lấy phần tử lớn nhất ở trong 2 hàng và 2 cột
model.add(MaxPooling2D(2,2))
# Duỗi thẳng dữ liệu 
model.add(Flatten())
# Tạo lớp ẩn thứ nhất với 128 tín hiệu ra
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
# Tạo lớp ẩn thứ hai với 8 tín hiệu ra
model.add(Dense(8,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                    

In [ ]:
# Biên dịch 
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Gán biến lại để vẽ đồ thị, với 100 lần học (epochs), mỗi lần học thì chỉ học␣128 dữ liệu (batch_size), khi sai số không thay đổi trong 100 lần học thì sẽ␣ dừng học 
history=model.fit(training_set,epochs=100,batch_size=128,verbose=1, validation_data=validation_set, callbacks=[EarlyStopping(monitor='val_loss', patience=100)])

Epoch 1/100
233/233 [==============================] - 3334s 14s/step - loss: 1.8511 - accuracy: 0.2838 - val_loss: 1.7252 - val_accuracy: 0.3042
Epoch 2/100
233/233 [==============================] - 153s 656ms/step - loss: 1.7317 - accuracy: 0.3060 - val_loss: 1.7021 - val_accuracy: 0.3205
Epoch 3/100
233/233 [==============================] - 151s 650ms/step - loss: 1.7079 - accuracy: 0.3190 - val_loss: 1.6931 - val_accuracy: 0.3142
Epoch 4/100
233/233 [==============================] - 152s 654ms/step - loss: 1.7021 - accuracy: 0.3182 - val_loss: 1.7119 - val_accuracy: 0.2941
Epoch 5/100
233/233 [==============================] - 152s 653ms/step - loss: 1.6731 - accuracy: 0.3314 - val_loss: 1.6680 - val_accuracy: 0.3132
Epoch 6/100
233/233 [==============================] - 152s 652ms/step - loss: 1.6328 - accuracy: 0.3421 - val_loss: 1.5999 - val_accuracy: 0.3487
Epoch 7/100
233/233 [==============================] - 152s 653ms/step - loss: 1.5795 - accuracy: 0.3525 - val_loss: 1.

In [ ]:
# Đánh giá độ chính xác của mô hình 
Score=model.evaluate(training_set,verbose=0)
print('Train Loss', Score[0])
print('Train Accuracy', Score[1])

In [ ]:
# Vẽ đồ thị giữa số lần học (Epochs) và độ chính xác (Accuracy)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train','Validation'])
plt.show

In [ ]:
model.save("project.h5")

In [ ]:
# Tải mô hình 
model_CNN=load_model('Project.h5')

In [ ]:
test="/content/drive/MyDrive/Colab_Notebooks/project/TEST"

for i in os.listdir(test):
  img=load_img(test+'/'+i,target_size=(150,150))
  plt.imshow(img)
  img=img_to_array(img)
  img=img.astype('float32')
  img=img/255
  img=np.expand_dims(img,axis=0)
  result=model_CNN.predict(img)
  if round(result[0][0])==1:
    prediction='THOÁI HOÁ'
  if round(result[0][1])==1:
    prediction='ĐỤC THUỶ TINH THỂ'
  if round(result[0][2])==1:
    prediction='TIỂU ĐƯỜNG'
  if round(result[0][3])==1:
    prediction='TĂNG NHÃN ÁP'
  if round(result[0][4])==1:
    prediction='TĂNG HUYẾT ÁP'
  if round(result[0][5])==1:
    prediction='BÌNH THƯỜNG'
  if round(result[0][6])==1:
    prediction='BỆNH LÝ KHÁC'
  if round(result[0][7])==1:
    prediction='CẬN THỊ'
  print(prediction)
  plt.show()